<a href="https://colab.research.google.com/github/Abhishekpamulapati/PortfolioProjects/blob/main/Pyspark/Mini_Transformation_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql.functions import *

Reading from open datasets on databricks

In [ ]:
fire_df = spark.read.format("csv")\
            .option("header", "true")\
            .option("inferSchema", "true")\
            .load("dbfs:/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv")
display(fire_df)

In [ ]:
fire_df.printSchema()

In [ ]:
fire_df=renamed_fire_df\
    .withColumn("CallDate",to_date("CallDate",'MM/dd/yyyy'))\
    .withColumn("WatchDate",to_date("WatchDate",'MM/dd/yyyy'))\
    .withColumn("AvailableDtTm",to_timestamp("AvailableDtTm","MM/dd/yyyy HH:mm:ss a"))\
    .withColumn("Delay",round("Delay",2))

In [ ]:
display(renamed_fire_df)
renamed_fire_df.printSchema()

In [ ]:
fire_df.cache()

Q1. How many distinct types of calls were made to the Fire department?
SELECT COUNT(DISTINCT callType) AS distinct_callTypes FROM demo_db.fire_service_tbl WHERE CallType IS NOT NULL

In [ ]:
fire_df.createOrReplaceTempView("fire_service_calls_tempview")
ql_sql_df = spark.sql("""
                      SELECT COUNT(DISTINCT callType) AS distinct_callTypes FROM fire_service_calls_tempview WHERE CallType IS NOT NULL
                      """)
display(ql_sql_df)

In [ ]:
ql_df = fire_df.where("CallType IS NOT NULL")\
            .select("CallType")\
            .distinct()
print(ql_df.count())

Q2. What were distinct types of calls made to the Fire Department?
SELECT DISTINCT callType AS distinct_callTypes FROM demo_db.fire_service_tbl WHERE CallType IS NOT NULL

In [ ]:
ql_df2=fire_df.where("CallType IS NOT NULL")\
            .select(expr("CallType as distinct_callTypes"))\
            .distinct()
display(ql_df2)

Q3. Find out all response for delayed times greater than 5 mins?
SELECT callNumber, Delay FROM demo_db.fire_service_tbl WHERE Delay > 5

In [ ]:
ql_df3 = fire_df.where("Delay > 5")\
                .select("callNumber","Delay")
display(ql_df3)

Q4. What were the most common call types?
SELECT callType, count(*) as count FROM demo_db.fire_service_tbl WHERE CallType is NOT NULL GROUP BY callType ORDER BY count DESC

In [ ]:
ql_df4=fire_df.where("CallType IS NOT NULL")\
    .select("callType")\
    .groupBy("callType")\
    .count()\
    .orderBy("count",ascending=False)
ql_df4.show()

Q5. What zip codes accounted for most common calls?
SELECT callType,zipCode, count(*) as count FROM demo_db.fire_service_tbl WHERE CallType is NOT NULL GROUP BY callType, Zipcode ORDER BY count DESC

In [ ]:
ql_df5= fire_df.where("CallType IS NOT NULL")\
     .select("callType","zipCode")\
     .groupBy("callType","zipCode")\
     .count()\
    .orderBy("count",ascending=False)
display(ql_df5)

Q6. What SF neighborhoods are in the zip codes 94102, 94103?
SELECT zipCode, neighborhood FROM demo_db.fire_service_tbl WHERE Zipcode in ('94102','94103')

In [ ]:
ql_df6=fire_df.where(
    (fire_df.ZipCode=='94102')&(fire_df.ZipCode=='94103'))\
        .select("ZipCode","Neighborhood")
display(ql_df6)

Q7. What was the sum of all call alarms, average, min, and max of the call response times?
SELECT sum(NumAlarms), avg(Delay),min(delay),max(delay) FROM demo_db.fire_service_tbl

In [ ]:
ql_df7=fire_df.agg(sum("NumAlarms"),avg("Delay"),min("Delay"),max("Delay"))

display(ql_df7)


Q8. How many distinct years of data is in the dataset?
SELECT DISTINCT year(to_date(CallDate,"mm/dd/yyyy")) AS year_num FROM demo_db.fire_service_tbl ORDER BY year_num

In [ ]:
ql_df8= fire_df.select(year("CallDate").alias(" distinct_year"))\
                .distinct()\
                .orderBy("CallDate")
display(ql_df8)

Q9. What week of the year in 2018 had the most fire calls?
SELECT DISTINCT weekofyear((to_date(CallDate,"mm/dd/yyyy"))) weekyear,count(*) AS count FROM demo_db.fire_service_tbl WHERE year(to_date(CallDate,"mm/dd/yyyy"))== 2018 GROUP BY weekyear ORDER BY count DESC

In [ ]:
ql_df9 = fire_df.select(weekofyear("CallDate").alias("weekyear")) \
                .where(year("CallDate") == 2018) \
                .groupBy("weekyear") \
                .count() \
                .distinct() \
                .orderBy("weekyear")

display(ql_df9)

Q10. What neighborhoods in SF had the worst response time in 2018?
SELECT neighborhood,delay FROM demo_db.fire_service_tbl WHERE year(to_date(CallDate,"mm/dd/yyyy"))==2018 ORDER BY Delay DESC

In [ ]:
fire_df.where("year(CallDate)==2018")\
        .select("neighborhood","Delay")\
        .orderBy("Delay",ascending=False)\
        .show()